

[lacentrale](https://www.lacentrale.fr/listing)



# Les voitures selon [lacentrale.fr](https://www.lacentrale.fr/listing)

<img src= ''>

L'objectif de ce projet est de collecter les données des voitures présents sur la page `https://www.lacentrale.fr/listing` du site lacentrale.fr.

Les données que nous collecterons seront les suivantes :
name
price
year
origin
registration_date
technical_inspection
first_hand
mileage
fuel_type
transmission
num_doors
num_seats
power
co2_emission
length
trunk_volume
critair_rating
combined_consumption

# Lacentrale

### Différentes Méthodes utilisées pour le scrapping

In [1]:
import requests
from lxml import etree
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
from tqdm import tqdm


# Function to fetch a page
def fetch_page(url, page=None):
    if page is not None:
        url = f"{url}?page={page}"
    with requests.Session() as session:
        response = session.get(url)
        soup = BeautifulSoup(response.content, features="lxml")
        return soup


# Function to extract details of a car
def extract_car_details(car_card):
    car = {
        "name": None,
        "price": None,
        "year": None,
        "origin": None,
        "registration_date": None,
        "technical_inspection": None,
        "first_hand": None,
        "mileage": None,
        "fuel_type": None,
        "transmission": None,
        "num_doors": None,
        "num_seats": None,
        "power": None,
        "co2_emission": None,
        "length": None,
        "trunk_volume": None,
        "critair_rating": None,
        "combined_consumption": None,
    }

    dom = etree.HTML(str(car_card))

    if (
        car_card.find(
            "div",
            class_="Text_Text_text SummaryInformation_title__5CYhW Text_Text_headline3",
        )
        != None
    ):
        car["name"] = car_card.find(
            "div",
            class_="Text_Text_text SummaryInformation_title__5CYhW Text_Text_headline3",
        ).text.strip()

    if car_card.find("span", class_="PriceInformation_classifiedPrice__b-Jae") != None:
        car["price"] = car_card.find(
            "span", class_="PriceInformation_classifiedPrice__b-Jae"
        ).text.strip()

    if len(dom.xpath('//*[@id="year"]/span[2]/span')) > 0:
        car["year"] = dom.xpath('//*[@id="year"]/span[2]/span')[0].text.strip()

    if len(dom.xpath('//*[@id="origin"]/span[2]/span[1]')) > 0:
        car["origin"] = dom.xpath('//*[@id="origin"]/span[2]/span[1]')[0].text.strip()

    if len(dom.xpath('//*[@id="firstCirculationDate"]/span[2]/span')) > 0:
        car["registration_date"] = dom.xpath(
            '//*[@id="firstCirculationDate"]/span[2]/span'
        )[0].text.strip()

    if len(dom.xpath('//*[@id="technicalControl"]/span[2]/span[1]')) > 0:
        car["technical_inspection"] = dom.xpath(
            '//*[@id="technicalControl"]/span[2]/span[1]'
        )[0].text.strip()

    if len(dom.xpath('//*[@id="firstHand"]/span[2]/span[1]')) > 0:
        car["first_hand"] = dom.xpath('//*[@id="firstHand"]/span[2]/span[1]')[
            0
        ].text.strip()

    if len(dom.xpath('//*[@id="mileage"]/span[2]/span[1]')) > 0:
        car["mileage"] = dom.xpath('//*[@id="mileage"]/span[2]/span[1]')[0].text.strip()

    if len(dom.xpath('//*[@id="energy"]/span[2]/span')) > 0:
        car["fuel_type"] = dom.xpath('//*[@id="energy"]/span[2]/span')[0].text.strip()

    if len(dom.xpath('//*[@id="gearbox"]/span[2]/span')) > 0:
        car["transmission"] = dom.xpath('//*[@id="gearbox"]/span[2]/span')[
            0
        ].text.strip()

    if len(dom.xpath('//*[@id="doors"]/span[2]/span')) > 0:
        car["num_doors"] = dom.xpath('//*[@id="doors"]/span[2]/span')[0].text.strip()

    if len(dom.xpath('//*[@id="seats"]/span[2]/span')) > 0:
        car["num_seats"] = dom.xpath('//*[@id="seats"]/span[2]/span')[0].text.strip()

    if len(dom.xpath('//*[@id="ratedHorsePower"]/span[2]/span[1]')) > 0:
        car["power"] = dom.xpath('//*[@id="ratedHorsePower"]/span[2]/span[1]')[
            0
        ].text.strip()

    if len(dom.xpath('//*[@id="co2"]/span[2]/div/div/div')) > 0:
        car["co2_emission"] = dom.xpath('//*[@id="co2"]/span[2]/div/div/div')[
            0
        ].text.strip()

    if len(dom.xpath('//*[@id="length"]/span[2]/span')) > 0:
        car["length"] = dom.xpath('//*[@id="length"]/span[2]/span')[0].text.strip()

    if (
        len(
            dom.xpath(
                '//*[@id="general-information"]/div/div[2]/div/div[2]/div[1]/section/ul/li[14]/div/button/span'
            )
        )
        > 0
    ):
        car["trunk_volume"] = dom.xpath(
            '//*[@id="general-information"]/div/div[2]/div/div[2]/div[1]/section/ul/li[14]/div/button/span'
        )[0].text.strip()

    if len(dom.xpath('//*[@id="critAir"]/span[2]/span[1]/span[1]')) > 0:
        car["critair_rating"] = dom.xpath('//*[@id="critAir"]/span[2]/span[1]/span[1]')[
            0
        ].text.strip()

    if len(dom.xpath('//*[@id="consumption"]/span[2]')) > 0:
        car["combined_consumption"] = dom.xpath('//*[@id="consumption"]/span[2]')[
            0
        ].text.strip()

    return car


# Function to get cars from a page
def get_cars_from_page(soup):
    cars = soup.find_all("div", class_=["searchCardContainer", "boostVo__container"])
    data = []
    for car in cars:
        car_url = urljoin(
            "https://www.lacentrale.fr",
            car.find("a", class_="Vehiculecard_Vehiculecard_vehiculeCard")["href"],
        )
        page = fetch_page(car_url)
        data.append(extract_car_details(page))
    return data

### Lancement du scrapping

In [2]:
# URL of the site to scrape
url = "https://www.lacentrale.fr/listing"
max_page = 500
cars = []

# Retrieve car details from all pages
for current_page in tqdm(range(1, max_page + 1)):
    try:
        page = fetch_page(url, str(current_page))
        car_data = get_cars_from_page(page)
        cars.extend(car_data)
    except requests.RequestException as e:
        print(f"Failed to fetch page {current_page}: {e}")
        pass


100%|██████████| 500/500 [1:02:27<00:00,  7.50s/it]


,name,price,year,origin,registration_date,technical_inspection,first_hand,mileage,fuel_type,transmission,num_doors,num_seats,power,co2_emission,length,trunk_volume,critair_rating,combined_consumption
0,NISSAN NAVARA II,36 880 €,2018,France,27/11/2018,Requis,Non,23 395 km,Diesel,Manuelle,4,5,8,E,"5,30 m",None,2,6
1,MINI MINI III 3P,18 620 €,2017,France,23/09/2017,Requis,Non,125 000 km,Diesel,Manuelle,3,4,5,A,"3,82 m",None,2,"3,1"
2,FORD PUMA 2,31 820 €,2024,Importé,19/07/2024,Non requis,Non,12 km,Hybride essence électrique,Manuelle,5,None,6,C,None,None,Euro 1 et avant,None
3,VOLKSWAGEN COCCINELLE phase 2,30 990 €,2020,France,03/12/2020,Non requis,Non,64 210 km,Essence,Automatique,2,4,8,C,"4,29 m",None,1,"4,9"
4,AUDI Q5 II SPORTBACK phase 2,82 860 €,2022,France,05/09/2022,Non requis,Oui,16 464 km,Hybride essence électrique,Automatique,5,5,16,A,"4,69 m",None,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8494,VOLVO XC40,38 410 €,2021,France,25/03/2021,Non requis,Non,33 185 km,Diesel,Automatique,5,5,8,D,"4,42 m",460,2,6
8495,PEUGEOT 208 II,21 380 €,2022,France,23/02/2022,Non requis,Non,13 274 km,Essence,Manuelle,5,5,5,B,"4,05 m",311,1,"5,3"
8496,PEUGEOT 308 III,41 730 €,2023,France,10/05/2023,Non requis,Oui,19 360 km,Diesel,Automatique,5,5,7,B,"4,37 m",412,2,"4,5"
8497,PEUGEOT RIFTER,28 870 €,2021,Importé,21/01/2022,Non requis,Non,65 383 km,Diesel,Manuelle,5,None,7,D,None,None,Euro 1 et avant,None


### Nettoyage et exploration de la data

In [68]:
# Create a DataFrame from the collected car data
df = pd.DataFrame(cars)
df.describe()

,name,price,year,origin,registration_date,technical_inspection,first_hand,mileage,fuel_type,transmission,num_doors,num_seats,power,co2_emission,length,trunk_volume,critair_rating,combined_consumption
count,8499,8499,8499,8499,8499,8499,8499,8499,8499,8499,8444,7725,8484,7965,7670,2409,8353,6097
unique,1247,3209,42,2,2808,2,2,5448,8,2,4,10,70,7,201,236,6,139
top,PEUGEOT 208 II,34 980 €,2022,France,24/03/2016,Non requis,Non,12 km,Essence,Automatique,5,5,7,C,"4,45 m",380,1,"5,1"
freq,188,77,1380,7350,70,6335,4985,197,3961,5228,6763,6154,1474,2595,329,111,4273,252


In [69]:
# 1. Convert Data Types
df['price'] = df['price'].str.replace('[^\d.]', '', regex=True).astype(float)
df['year'] = df['year'].astype(int)
df['registration_date'] = pd.to_datetime(df['registration_date'])
df['mileage'] = df['mileage'].str.replace(' km', '').str.replace(' ', '').astype(float)
df["length"] = (
    df["length"]
    .str.replace(" m", "")
    .str.replace(",", ".")
    .str.replace(" ", "")
    .astype(float)
)

# TODO: change power to the regular one
# # df["power"] = df["power"].str.replace(" ch", "").str.replace(" ", "").astype(float)

df.info()
# df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8499 entries, 0 to 8498
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   name                  8499 non-null   object        
 1   price                 8499 non-null   float64       
 2   year                  8499 non-null   int64         
 3   origin                8499 non-null   object        
 4   registration_date     8499 non-null   datetime64[ns]
 5   technical_inspection  8499 non-null   object        
 6   first_hand            8499 non-null   object        
 7   mileage               8499 non-null   float64       
 8   fuel_type             8499 non-null   object        
 9   transmission          8499 non-null   object        
 10  num_doors             8444 non-null   object        
 11  num_seats             7725 non-null   object        
 12  power                 8484 non-null   object        
 13  co2_emission      

/var/folders/1p/wfwrd_4x7f179w6rjrl3xkw00000gn/T/ipykernel_4915/2275532161.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['registration_date'] = pd.to_datetime(df['registration_date'])


In [70]:
# 2. Handle Missing Values
df.fillna(
    {
        "num_doors": df["num_doors"].mode()[0],
        "num_seats": df["num_seats"].mode()[0],
        "power": df["power"].mode()[0],
        "co2_emission": df["co2_emission"].mode()[0],
        "length": df["length"].mode()[0],
        "critair_rating": "Unknown",
        "combined_consumption": df["combined_consumption"].mode()[0],
    },
    inplace=True,
)

df["num_doors"] = df["num_doors"].astype(int)
df["num_seats"] = df["num_seats"].astype(int)
df.describe()

,price,year,registration_date,mileage,num_doors,num_seats,length
count,8.499000e+03,8499.000000,8499,8499.000000,8499.000000,8499.000000,8499.000000
mean,4.022002e+04,2020.256971,2020-10-31 11:40:25.838333952,54919.364160,4.625603,4.862337,4.452425
min,3.400000e+03,1960.000000,1960-06-02 00:00:00,1.000000,2.000000,2.000000,0.430000
25%,2.067000e+04,2019.000000,2019-09-25 00:00:00,19499.000000,5.000000,5.000000,4.210000
50%,2.993000e+04,2021.000000,2021-06-10 00:00:00,46778.000000,5.000000,5.000000,4.450000
75%,4.282000e+04,2022.000000,2023-01-14 12:00:00,80977.500000,5.000000,5.000000,4.600000
max,1.078670e+06,2024.000000,2025-01-31 00:00:00,380720.000000,5.000000,65.000000,148.300000
std,4.387946e+04,4.072865,NaN,45198.820164,0.846175,1.224838,2.180962


In [71]:
from sklearn.preprocessing import LabelEncoder

In [72]:
# # 4. Encode Categorical Variables
label_encoder = LabelEncoder()
df["fuel_type"] = label_encoder.fit_transform(df["fuel_type"])
df["origin"] = label_encoder.fit_transform(df["origin"])
df["transmission"] = label_encoder.fit_transform(df["transmission"])
df["technical_inspection"] = label_encoder.fit_transform(df["technical_inspection"])
df["first_hand"] = label_encoder.fit_transform(df["first_hand"])
df["co2_emission"] = label_encoder.fit_transform(df["co2_emission"])
df["critair_rating"] = label_encoder.fit_transform(df["critair_rating"])

df.describe()

,price,year,origin,registration_date,technical_inspection,first_hand,mileage,fuel_type,transmission,num_doors,num_seats,co2_emission,length,critair_rating
count,8.499000e+03,8499.000000,8499.000000,8499,8499.000000,8499.000000,8499.000000,8499.000000,8499.000000,8499.000000,8499.000000,8499.000000,8499.000000,8499.000000
mean,4.022002e+04,2020.256971,0.135192,2020-10-31 11:40:25.838333952,0.254618,0.413460,54919.364160,4.436169,0.384869,4.625603,4.862337,1.941405,4.452425,1.667961
min,3.400000e+03,1960.000000,0.000000,1960-06-02 00:00:00,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.430000,0.000000
25%,2.067000e+04,2019.000000,0.000000,2019-09-25 00:00:00,0.000000,0.000000,19499.000000,3.000000,0.000000,5.000000,5.000000,1.000000,4.210000,1.000000
50%,2.993000e+04,2021.000000,0.000000,2021-06-10 00:00:00,0.000000,0.000000,46778.000000,5.000000,0.000000,5.000000,5.000000,2.000000,4.450000,1.000000
75%,4.282000e+04,2022.000000,0.000000,2023-01-14 12:00:00,1.000000,1.000000,80977.500000,5.000000,1.000000,5.000000,5.000000,3.000000,4.600000,2.000000
max,1.078670e+06,2024.000000,1.000000,2025-01-31 00:00:00,1.000000,1.000000,380720.000000,7.000000,1.000000,5.000000,65.000000,6.000000,148.300000,6.000000
std,4.387946e+04,4.072865,0.341949,NaN,0.435672,0.492483,45198.820164,1.283259,0.486593,0.846175,1.224838,1.430792,2.180962,1.267615


In [73]:
# 5. Drop Unnecessary Columns
df.drop(["trunk_volume"], axis=1, inplace=True)

In [77]:
# Display the cleaned DataFrame
df.tail()

,name,price,year,origin,registration_date,technical_inspection,first_hand,mileage,fuel_type,transmission,num_doors,num_seats,power,co2_emission,length,critair_rating,combined_consumption
8494,VOLVO XC40,38410.0,2021,0,2021-03-25,0,0,33185.0,3,0,5,5,8,3,4.42,2,6
8495,PEUGEOT 208 II,21380.0,2022,0,2022-02-23,0,0,13274.0,5,1,5,5,5,1,4.05,1,"5,3"
8496,PEUGEOT 308 III,41730.0,2023,0,2023-05-10,0,1,19360.0,3,0,5,5,7,1,4.37,2,"4,5"
8497,PEUGEOT RIFTER,28870.0,2021,1,2022-01-21,0,0,65383.0,3,1,5,5,7,3,4.45,5,"5,1"
8498,KIA XCEED phase 2,46150.0,2024,0,2025-01-30,0,0,7410.0,7,0,5,5,5,0,4.39,1,"5,1"


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8499 entries, 0 to 8498
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   name                  8499 non-null   object        
 1   price                 8499 non-null   float64       
 2   year                  8499 non-null   int64         
 3   origin                8499 non-null   int64         
 4   registration_date     8499 non-null   datetime64[ns]
 5   technical_inspection  8499 non-null   int64         
 6   first_hand            8499 non-null   int64         
 7   mileage               8499 non-null   float64       
 8   fuel_type             8499 non-null   int64         
 9   transmission          8499 non-null   int64         
 10  num_doors             8499 non-null   int64         
 11  num_seats             8499 non-null   int64         
 12  power                 8499 non-null   object        
 13  co2_emission      

### Enregistrement en base

In [75]:
# api_url = "http://localhost:8000/api/car_data/"

# # Convert the DataFrame to a list of dictionaries
# car_data_list = df.to_dict(orient="records")

# # Send a POST request to the API endpoint
# try:
#     response = requests.post(api_url, json=car_data_list)
#     if response.status_code == 201:
#         print("Data successfully posted to the API. 🚀")
#     else:
#         print("Failed to post data. Status code:", response.status_code)
# except requests.RequestException as e:
#     print("An error occurred while sending the POST request:", str(e))